In [2]:
import time
import pandas as pd

pd.set_option('display.float_format', '{:.0f}'.format)

tickers = ["MSFT", "AMZN", "GOOGL", "GOOG", "TSLA", "BRK-B", "NVDA", "META", "UNH", "AAPL"]

df = pd.DataFrame()

for ticker in tickers:
    data = pd.read_csv(f"{ticker}.csv")
    data['ticker']= ticker
    df=pd.concat([df, data], ignore_index=True)

df.to_csv("merged_financial_data.csv")


In [3]:
# Parse date and sort

pd.read_csv("merged_financial_data.csv")
df["fiscalDateEnding"] = pd.to_datetime(df["fiscalDateEnding"])
df = df.sort_values(by=["ticker", "fiscalDateEnding"]).reset_index(drop=True)

df.head(-1)


,date,fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,...,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome,ticker
0,1970-01-01 00:00:00.000000080,2005-03-31,USD,968000000,3243000000,2275000000,2275000000,402000000,447000000,119000000,...,41000000,435000000,145000000,NaN,0,NaN,402000000,443000000,290000000,AAPL
1,1970-01-01 00:00:00.000000079,2005-06-30,USD,1044000000,3520000000,2476000000,2476000000,426000000,472000000,145000000,...,46000000,472000000,153000000,NaN,0,NaN,426000000,472000000,319000000,AAPL
2,1970-01-01 00:00:00.000000078,2005-09-30,USD,1035000000,3678000000,2643000000,2643000000,418000000,470000000,147000000,...,51000000,478000000,48000000,NaN,0,NaN,418000000,469000000,430000000,AAPL
3,1970-01-01 00:00:00.000000077,2005-12-31,USD,1564000000,5749000000,4185000000,4185000000,750000000,632000000,182000000,...,52000000,831000000,266000000,NaN,0,NaN,750000000,802000000,565000000,AAPL
4,1970-01-01 00:00:00.000000076,2006-03-31,USD,1297000000,4359000000,3062000000,3062000000,529000000,592000000,176000000,...,50000000,605000000,195000000,NaN,0,NaN,529000000,579000000,410000000,AAPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
773,1970-01-01 00:00:00.000000005,2023-12-31,USD,20823000000,93248000000,72425000000,72425000000,7689000000,NaN,NaN,...,974000000,6859000000,1184000000,NaN,5675000000,NaN,6510000000,7484000000,5455000000,UNH
774,1970-01-01 00:00:00.000000004,2024-03-31,USD,21994000000,98785000000,76791000000,76791000000,7931000000,NaN,NaN,...,997000000,1000000,1222000000,NaN,-1221000000,NaN,845000000,1842000000,-1409000000,UNH
775,1970-01-01 00:00:00.000000003,2024-06-30,USD,21060000000,97858000000,76798000000,76798000000,7875000000,NaN,NaN,...,1020000000,5665000000,1244000000,NaN,4421000000,NaN,6650000000,7670000000,4216000000,UNH
776,1970-01-01 00:00:00.000000002,2024-09-30,USD,21386000000,99177000000,77791000000,77791000000,8708000000,NaN,NaN,...,1041000000,7614000000,1356000000,NaN,6258000000,NaN,8688000000,9729000000,6055000000,UNH


In [4]:

# Helper function
def to_float(x):
    return 0.0 if pd.isna(x) or x == 'None' or x=="NaN" else float(x)


In [5]:
# Feature engineering
#opratingIncome = Operating Profit/(Loss)
#Profit/Loss Before Tax = incomeBeforeTax
#Tax = IncomeTaxExpense
#Profit/Loss After Tax = netIncome
df["nonOperatingItems"] = (
    df["interestIncome"].apply(to_float) +
    df["investmentIncomeNet"].apply(to_float) +
    df["otherNonOperatingIncome"].apply(to_float) +
    df["nonInterestIncome"].apply(to_float) -
    df["interestExpense"].apply(to_float) -
    df["interestAndDebtExpense"].apply(to_float)
)

# df["Net Interest"] = df["interestIncome"].apply(to_float) - df["interestExpense"].apply(to_float)

df["netInterest"] = df.apply(
    lambda row: to_float(row["netInterestIncome"])
    if pd.notna(row["netInterestIncome"])
    else to_float(row["interestIncome"]) - to_float(row["interestExpense"]),
    axis=1
)
df.head(-1)



,date,fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,...,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome,ticker,nonOperatingItems,netInterest
0,1970-01-01 00:00:00.000000080,2005-03-31,USD,968000000,3243000000,2275000000,2275000000,402000000,447000000,119000000,...,145000000,NaN,0,NaN,402000000,443000000,290000000,AAPL,-33000000,-33000000
1,1970-01-01 00:00:00.000000079,2005-06-30,USD,1044000000,3520000000,2476000000,2476000000,426000000,472000000,145000000,...,153000000,NaN,0,NaN,426000000,472000000,319000000,AAPL,-46000000,-46000000
2,1970-01-01 00:00:00.000000078,2005-09-30,USD,1035000000,3678000000,2643000000,2643000000,418000000,470000000,147000000,...,48000000,NaN,0,NaN,418000000,469000000,430000000,AAPL,-60000000,-60000000
3,1970-01-01 00:00:00.000000077,2005-12-31,USD,1564000000,5749000000,4185000000,4185000000,750000000,632000000,182000000,...,266000000,NaN,0,NaN,750000000,802000000,565000000,AAPL,-81000000,-81000000
4,1970-01-01 00:00:00.000000076,2006-03-31,USD,1297000000,4359000000,3062000000,3062000000,529000000,592000000,176000000,...,195000000,NaN,0,NaN,529000000,579000000,410000000,AAPL,-76000000,-76000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
773,1970-01-01 00:00:00.000000005,2023-12-31,USD,20823000000,93248000000,72425000000,72425000000,7689000000,NaN,NaN,...,1184000000,NaN,5675000000,NaN,6510000000,7484000000,5455000000,UNH,-830000000,-830000000
774,1970-01-01 00:00:00.000000004,2024-03-31,USD,21994000000,98785000000,76791000000,76791000000,7931000000,NaN,NaN,...,1222000000,NaN,-1221000000,NaN,845000000,1842000000,-1409000000,UNH,-844000000,-844000000
775,1970-01-01 00:00:00.000000003,2024-06-30,USD,21060000000,97858000000,76798000000,76798000000,7875000000,NaN,NaN,...,1244000000,NaN,4421000000,NaN,6650000000,7670000000,4216000000,UNH,-985000000,-985000000
776,1970-01-01 00:00:00.000000002,2024-09-30,USD,21386000000,99177000000,77791000000,77791000000,8708000000,NaN,NaN,...,1356000000,NaN,6258000000,NaN,8688000000,9729000000,6055000000,UNH,-1074000000,-1074000000


In [6]:
df = df[["fiscalDateEnding","ticker","operatingIncome","ebit","incomeBeforeTax","incomeTaxExpense","netIncome","nonOperatingItems","netInterest"]]
df.to_csv("preprocessed_merged_financial_data.csv")
df.head(-1)


,fiscalDateEnding,ticker,operatingIncome,ebit,incomeBeforeTax,incomeTaxExpense,netIncome,nonOperatingItems,netInterest
0,2005-03-31,AAPL,402000000,402000000,435000000,145000000,290000000,-33000000,-33000000
1,2005-06-30,AAPL,426000000,426000000,472000000,153000000,319000000,-46000000,-46000000
2,2005-09-30,AAPL,418000000,418000000,478000000,48000000,430000000,-60000000,-60000000
3,2005-12-31,AAPL,750000000,750000000,831000000,266000000,565000000,-81000000,-81000000
4,2006-03-31,AAPL,529000000,529000000,605000000,195000000,410000000,-76000000,-76000000
...,...,...,...,...,...,...,...,...,...
773,2023-12-31,UNH,7689000000,6510000000,6859000000,1184000000,5455000000,-830000000,-830000000
774,2024-03-31,UNH,7931000000,845000000,1000000,1222000000,-1409000000,-844000000,-844000000
775,2024-06-30,UNH,7875000000,6650000000,5665000000,1244000000,4216000000,-985000000,-985000000
776,2024-09-30,UNH,8708000000,8688000000,7614000000,1356000000,6055000000,-1074000000,-1074000000
